In [1]:
using Random
import Base:+,-,*,println, sum, broadcasted, size, adjoint, show, dropdims, tanh
using Base.Iterators:partition, flatten

mutable struct Tensor
    data
    autograd
    creators
    creation_op
    id
    children
    grad 
    index_select_indices
    softmax_output
    target_dist
    
    function Tensor(data; autograd=false, creators=nothing, creation_op = nothing, id=nothing)
        if isnothing(id)
            id = rand(1:100000)
        end
        T = new(data, autograd, creators, creation_op, id)
        T.children = Dict()
        T.grad = nothing
        T.index_select_indices = nothing
        
        if !(isnothing(creators))
            for c in creators
                if haskey(c.children, T.id)
                    c.children[T.id] += 1
                else
                    c.children[T.id] = 1
                end
            end
        end
        return T
    end
end

function all_children_grads_accounted_for(t::Tensor)
    for (id, cnt) in t.children
        if (cnt != 0)
            return false
        end
    end
    return true
end

function backward(t::Tensor, grad=nothing, grad_origin=nothing)
    if t.autograd
        if isnothing(grad)
            grad = Tensor(ones(size(t.data)))
        end
    
        if !(isnothing(grad_origin))
            if t.children[grad_origin.id] == 0
                return
                throw("cannot backprop more than once")
            else
                t.children[grad_origin.id] -= 1
            end
        end
        
        if isnothing(t.grad)
            t.grad = grad
        else
            t.grad += grad
        end
        
        # grads must not have grads of their own
        @assert !grad.autograd
        
        # only continue backpropping if there's something to
        # backprop into and if all gradients (from children)
        # are accounted for override waiting for children if
        # "backprop" was called on this variable directly
        
        if (!isnothing(t.creators) && (all_children_grads_accounted_for(t) || isnothing(grad_origin)))
            if t.creation_op == "add"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], t.grad, t)
            end
            
            if t.creation_op == "sub"
                backward(t.creators[1], t.grad, t)
                backward(t.creators[2], -t.grad, t)
            end
            
            if t.creation_op == "mul"
                new_ = t.grad .* t.creators[2]
                backward(t.creators[1], new_, t)
                new_ = t.grad .* t.creators[1]
                backward(t.creators[2], new_, t)
            end
            
            if t.creation_op == "mm"
                c1 = t.creators[1]
                c2 = t.creators[2]
                new_ =  t.grad * c2' ################
                backward(c1, new_)
                new_ = c1' * t.grad
                backward(c2, new_)
            end
                  
            if t.creation_op == "transpose"
                backward(t.creators[1], t.grad')
            end
            
            if occursin("sum", t.creation_op)
                dim = parse(Int, split(t.creation_op, "_")[2])
                backward(t.creators[1], expand(t.grad, dim, size(t.creators[1].data)[dim]))
            end
            
            if occursin("expand", t.creation_op)
                dim = parse(Int, split(t.creation_op, "_")[2])
                ndims_cr = ndims(t.creators[1].data)
                backward(t.creators[1], dropdims(sum(t.grad;dims=dim);dims=dim, ndims_cr=ndims_cr))
            end
            
            if t.creation_op == "neg"
                backward(t.creators[1], -t.grad)
            end
            
            if t.creation_op == "sigmoid"
                ones_ = Tensor(ones(size(t.grad.data)))
                backward(t.creators[1], t.grad .* t .* (ones_ - t) )
            end
            
            if t.creation_op == "tanh"
                ones_ = Tensor(ones(size(t.grad.data)))
                backward(t.creators[1], t.grad .* (ones_ - (t .* t)))
            end
            
            if t.creation_op == "index_select"
                new_grad = zeros(size(t.creators[1]))
                indices = t.index_select_indices.data
                major_chunks = partition(1:size(t.grad,2),length(indices))
                grad_chunks = [t.grad.data[:,inds][:,j]  for(i,inds) in enumerate(major_chunks) for j=1:size(inds)[1]]
    
                for (i,ind) in enumerate(flatten(indices))
                    new_grad[:,ind] +=  grad_chunks[i]
                end
                backward(t.creators[1], Tensor(new_grad))
            end
            if t.creation_op == "cross_entropy"
                dx = t.softmax_output .- t.target_dist
                backward(t.creators[1], Tensor(dx))
            end
        end
    end
end
                        
size(a::Tensor) = size(a.data)
size(a::Tensor, ind::Int) = size(a.data, ind)

function +(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data + b.data; autograd=true, creators=[a,b], creation_op = "add")
    end
    return Tensor(a.data+b.data)
end

function -(a::Tensor)
    if (a.autograd)
        return Tensor(a.data .* -1; autograd=true, creators=[a], creation_op = "neg")
    end
    return Tensor(a.data .* -1)
end

function -(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data - b.data; autograd=true, creators=[a,b], creation_op = "sub")
    end
    return Tensor(a.data-b.data)
end

#element-wise multiplication
function broadcasted(f::typeof(*), a::Tensor, b::Tensor)
    new_data = zeros(size(a.data))
    for i=1:length(new_data)
        new_data[i] = f(a.data[i] ,b.data[i])
    end
    if (a.autograd && b.autograd)
        return Tensor(new_data; autograd=true, creators=[a,b], creation_op ="mul")
    end
    return Tensor(new_data)
end

function broadcasted(f::typeof(-), a::Tensor, b::Tensor)
    new_data = zeros(size(a.data))
    for i=1:length(new_data)
        new_data[i] = -(a.data[i] ,b.data[i])
    end
    if (a.autograd && b.autograd)
        return Tensor(new_data; autograd=true, creators=[a,b], creation_op ="sub")
    end
    return Tensor(new_data)
end

function sum(a::Tensor; dims=dims)
    new_ = dropdims(sum(a.data ;dims=dims), dims = tuple(findall(size(a) .== 1)...))
    if (a.autograd)
        return Tensor(new_; autograd=true, creators=[a], creation_op = "sum_"*string(dims))
    end
    return Tensor(new_)
end

function dropdims(a::Tensor;dims=dims,ndims_cr=ndims_cr)
    if ndims(a.data) == ndims_cr
        return a
    end
    if (a.autograd)
        return Tensor(dropdims(a.data ;dims=dims); autograd=true, creators=[a], creation_op = "dropdims")
    end
    return Tensor(dropdims(a.data ;dims=dims))
end

function expand(a::Tensor, dim, copies)
    sz = size(a)
    rep = ntuple(d->d==dim ? copies : 1, length(sz)+1)
    new_size = ntuple(d->d<dim ? sz[d] : d == dim ? 1 : sz[d-1], length(sz)+1)
    new_data =  repeat(reshape(a.data, new_size), outer=rep)
    if (a.autograd)
        return Tensor(new_data; autograd=true, creators=[a], creation_op = "expand_"*string(dim))
    end
    return Tensor(new_data)
end

#transpose
function adjoint(a::Tensor)
    if (a.autograd)
        return Tensor(a.data';autograd=true, creators=[a], creation_op = "transpose")
    end
    return Tensor(a.data')
end

#matrix multiply 
function *(a::Tensor, b::Tensor)
    if (a.autograd && b.autograd)
        return Tensor(a.data * b.data; autograd=true, creators=[a,b], creation_op = "mm")
    end
    return Tensor(a.data * b.data)
end


function index_select_helper(a::Array, indices)
    return reduce(hcat,map(ind -> a[:,ind], indices))
end

function index_select(a::Tensor, indices::Tensor)
    new_ = index_select_helper(a.data, indices.data)
    if (a.autograd)
        T = Tensor(new_, autograd=true, creators=[a], creation_op = "index_select")
        T.index_select_indices = indices
        return T
    end
    return Tensor(new_)
end

println(t::Tensor) = println(t.data)
show(io::IO,m::MIME"text/plain",a::Tensor) = show(io,m,a.data)
                        
abstract type Layer end

function get_parameters(l::Layer)
    return l.parameters
end

mutable struct Linear <: Layer
    W
    b
    use_bias
    parameters
                            
    function Linear(n_inputs, n_outputs;bias=true)
        linear = new()
        linear.use_bias = bias
        linear.W = Tensor(randn(n_outputs, n_inputs) .* sqrt(2.0/n_inputs), autograd=true)
        if bias
            linear.b = Tensor(zeros(n_outputs), autograd=true) 
            linear.parameters = [linear.W,linear.b]
        else
            linear.parameters = [linear.W]
        end
        return linear
    end
end

function forward(l::Linear, input)
    if l.use_bias
        return (l.W * input)  + expand(l.b,2,size(input.data, 2))
    end
    return l.W * input
end                        

                        
mutable struct Sequential <: Layer
    layers
    function Sequential(layers)
        return new(layers)
    end
end

function add(s::Sequential, layer)
    push!(s.layers, layer)
end

function forward(s::Sequential, input)
    for layer in s.layers
        input = forward(layer, input)
    end
    return input
end

function get_parameters(s::Sequential)
    parameters = [get_parameters(layer) for layer in s.layers]
    return collect(Iterators.flatten(parameters))
end

mutable struct SGD
    parameters
    alpha
    SGD(parameters, alpha) = new(parameters, alpha)
end

function zero!(opt::SGD)
    for p in opt.parameters
        p.grad.data .*= 0.0
    end
end

function step(opt::SGD, zero=true)
    for p in opt.parameters
        p.data -= (p.grad.data .* opt.alpha)
        if zero
            p.grad.data .*= 0.0
        end
    end
end
                        
σ(x) = 1/(1+exp(-x))                        

struct Tanh <: Layer
    Tanh() = new()
end

struct Sigmoid <: Layer
    Sigmoid() = new()
end

function get_parameters(l::Tanh)
    return []
end

function get_parameters(l::Sigmoid)
    return []
end

function forward(l::Sigmoid, a::Tensor)
    if a.autograd
        return Tensor(σ.(a.data); autograd=true, creators=[a], creation_op = "sigmoid")
    end
    return Tensor(σ.(a.data))
end
        
function forward(l::Tanh, a::Tensor)
    if a.autograd
        return Tensor(tanh.(a.data); autograd=true, creators=[a], creation_op = "tanh")
    end
    return Tensor(tanh.(a.data))
end    
                        
                        
mutable struct Embedding <: Layer
    vocab_size
    dim
    weight
    parameters
    # this random initialiation style is just a convention from word2vec
    function Embedding(dim, vocab_size) 
        E = new(vocab_size, dim, Tensor((randn(dim, vocab_size) .- 0.5) ./ dim; autograd=true))
        E.parameters = [E.weight]
        return E
    end
end

function forward(E::Embedding, indices)
    return index_select(E.weight, indices)
end
                        
using Statistics: mean
using LinearAlgebra: I
function softmax(x)
    temp = exp.(x)
    return temp ./ sum(temp;dims=1)
end

struct CrossEntropyLoss 
    CrossEntropyLoss() = new()
end

function forward(l::CrossEntropyLoss, a::Tensor, target::Tensor)
    softmax_output = softmax(a.data)
    log_out = log.(softmax_output)
    sz = size(a.data, 1)
    identity = 1.0 .* Matrix(I, (sz, sz))
    target_dist = reshape(identity[:,target.data],(size(a.data)))
    loss = -mean(sum(log_out .* target_dist;dims=1))
    if a.autograd
        loss = Tensor(loss; autograd=true, creators=[a], creation_op = "cross_entropy")
        loss.softmax_output = softmax_output
        loss.target_dist = target_dist
        return loss
    end
    return Tensor(loss)
end


mutable struct RNNCell <: Layer
    n_hidden
    
    activation
    
    w_ih
    w_hh
    w_ho
    
    parameters
    
    function RNNCell(n_inputs, n_hidden, n_output, activation="sigmoid")
        if activation == "sigmoid"
            act = Sigmoid()
        elseif activation == "tanh"
            act = Tanh()
        else
            throw("Non-linearity not found")
        end
        
        parameters = []

        w_ih = Linear(n_inputs, n_hidden)
        w_hh = Linear(n_hidden, n_hidden)
        w_ho = Linear(n_hidden, n_output)
        
        push!(parameters, get_parameters(w_ih))
        push!(parameters, get_parameters(w_hh))
        push!(parameters, get_parameters(w_ho))
        parameters = collect(Iterators.flatten(parameters))
        return new(n_hidden, act, w_ih, w_hh, w_ho, parameters)
    end
end

function forward(rnn::RNNCell, input::Tensor, hidden::Tensor)
    from_prev_hidden = forward(rnn.w_hh, hidden)
    combined = forward(rnn.w_ih, input) + from_prev_hidden
    new_hidden = forward(rnn.activation, combined)
    output = forward(rnn.w_ho, new_hidden)
    return output, new_hidden
end

function init_hidden(rnn::RNNCell; batch_size=1)
    return Tensor(zeros(rnn.n_hidden, batch_size), autograd=true)
end

mutable struct LSTMCell <: Layer
    
    n_hidden
    
    xf
    xi
    xo
    xc
    
    hf
    hi
    ho
    hc
    
    w_ho
    parameters
    sigmoid
    tanh
    
    function LSTMCell(n_inputs, n_hidden, n_output)

        xf = Linear(n_inputs, n_hidden)
        xi = Linear(n_inputs, n_hidden)
        xo = Linear(n_inputs, n_hidden)        
        xc = Linear(n_inputs, n_hidden) 
        
        hf = Linear(n_hidden, n_hidden; bias=false)
        hi = Linear(n_hidden, n_hidden; bias=false)
        ho = Linear(n_hidden, n_hidden; bias=false)
        hc = Linear(n_hidden, n_hidden; bias=false) 
        
        w_ho = Linear(n_hidden, n_output; bias=false)
        
        parameters = [get_parameters(i) for i in [xf, xi, xo, xc, hf, hi, hc, w_ho]]
        parameters = collect(Iterators.flatten(parameters))
        
        return new(n_hidden, xf, xi, xo, xc, hf, hi, ho, hc, w_ho, parameters, Sigmoid(), Tanh())
    end
end

function forward(lstm::LSTMCell, input::Tensor, hidden)
    
    prev_hidden = hidden[1]        
    prev_cell = hidden[2]
    
    f = forward(lstm.xf, input) + forward(lstm.sigmoid, forward(lstm.hf, prev_hidden))
    i = forward(lstm.xi, input) + forward(lstm.sigmoid, forward(lstm.hi, prev_hidden))
    o = forward(lstm.xo, input) + forward(lstm.sigmoid, forward(lstm.ho, prev_hidden))
    g = forward(lstm.xc, input) + forward(lstm.tanh, forward(lstm.hc, prev_hidden))
    
    c = (f .* prev_cell) + (i .* g)

    h = o .* forward(lstm.tanh, c)
    
    output = forward(lstm.w_ho, h)
    
    return output, (h, c)
end

function init_hidden(lstm; batch_size=1)
    init_hidden = Tensor(zeros(lstm.n_hidden, batch_size), autograd=true)
    init_cell   = Tensor(zeros(lstm.n_hidden, batch_size), autograd=true)
    
    init_hidden.data[1,:] .+= 1
    init_cell.data[1,:] .+= 1
    return (init_hidden, init_cell)
end

init_hidden (generic function with 2 methods)

# Part 1: RNN Character Language Model

In [2]:
raw = read("shakespear.txt", String)
vocab = collect(Set(raw))
word2index = Dict()
for (i,word) in enumerate(vocab)
    word2index[word]=i
end

using Random:seed!;seed!(0)
embed = Embedding(512, length(vocab))
model = LSTMCell(512, 512, length(vocab))
model.w_ho.W.data .*= 0

criterion = CrossEntropyLoss()
optim = SGD(cat(get_parameters(model), get_parameters(embed); dims=1), 0.05);

function generate_sample(;n=30,init_char=' ')
    s = ""
    hidden = init_hidden(model, batch_size=1)
    input = Tensor([word2index[init_char]])
    for i=1:n
        rnn_input = forward(embed, input)
        output, hidden = forward(model, rnn_input, hidden)
        
        m = argmax(output.data).I[1]
        c = vocab[m]
        input = Tensor([m])
        s *= c
    end
    return s
end

indices = map(x->word2index[x], collect(raw))

batch_size = 16
bptt = 25
n_batches = trunc(Int,size(indices,1)/batch_size)
trimmed_indices = indices[1:n_batches*batch_size]

input_batched_indices = reshape(indices[1:n_batches*batch_size], (batch_size, n_batches))
target_batched_indices = reshape(indices[2:n_batches*batch_size+1], (batch_size, n_batches))

n_bptt = trunc(Int,((n_batches-1) / bptt))

input_batches = permutedims(reshape(input_batched_indices[:,1:n_bptt*bptt], (n_bptt, bptt, batch_size)), (3,1,2))
input_batches = reshape(input_batches, (batch_size,bptt,n_bptt))

target_batches = permutedims(reshape(target_batched_indices[:,1:n_bptt*bptt], (n_bptt, bptt, batch_size)), (3,1,2))
target_batches = reshape(target_batches, (batch_size,bptt,n_bptt))
min_loss = 1000

1000

In [3]:
function train(iterations=400;batch_size=16,bptt=25,min_loss=1000)
    for iter=1:iterations
        
        total_loss = 0
        n_loss = 0
        
        hidden = init_hidden(model, batch_size=batch_size)
        
        for batch_i=1:size(input_batches,3)
            losses = []
            hidden = [Tensor(hidden[1].data, autograd=true), Tensor(hidden[2].data, autograd=true)]

            for t=1:bptt
                input = Tensor(input_batches[:,t,batch_i], autograd=true)
                rnn_input = forward(embed, input)
                output, hidden = forward(model, rnn_input, hidden)
                
                target = Tensor(target_batches[:,t,batch_i], autograd=true)
                batch_loss = forward(criterion, output, target)
                
                if t==1
                    push!(losses, batch_loss)
                else
                    push!(losses, batch_loss + losses[end])
                end
            end
            
            loss = losses[end]
            backward(loss)
            step(optim)
            total_loss += loss.data/bptt
            
            epoch_loss = exp(total_loss / batch_i)
            
            if(epoch_loss < min_loss)
                min_loss = epoch_loss
#                 println()
            end
            
            log = "Iter: $(iter)"
            log *= " - Alpha: $(round(optim.alpha;digits=5))"
            log *= " - Batch $(batch_i)/$(size(input_batches,3))"
            log *= " - Min Loss: $(string(min_loss)[1:5])"
            log *= " - Loss: $(epoch_loss)"
            if batch_i == 1
                log*= " - " * replace(generate_sample(n=70, init_char='\n'),'\n' => " ")
            end
            if ((batch_i-1) % 10 == 0) || batch_i == size(input_batches,3)
                print(log,"\r")
            end
            
        end
        println()
        optim.alpha *= 0.99
    end
end

train (generic function with 2 methods)

In [ ]:
train(100)